Initialization

In [1]:
from scipy.optimize import minimize
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp,Operator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qiskit.quantum_info import Statevector,random_statevector
import math
import json
import os
import random

GREEN = "\033[92m"
RED = "\033[91m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
CYAN = "\033[96m"
MAGENTA = "\033[95m"
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
STANDARD = "\033[0m"

%run config.py

Load the directories and identify the project inside them.

In [2]:
def rotfitness(parameters, *args):
	qbits=int(len(parameters)/3)

	qc = QuantumCircuit(qbits)
	for i in range(qbits):
		qc.rx(float(parameters[3*i]), i)
		qc.ry(float(parameters[3*i+1]), i)
		qc.rz(float(parameters[3*i+2]), i)
	
	# print (qc.draw())
	qc=qc.reverse_bits()

	complexArray = []
	complexArray.append(complex(1,0))
	for i in range(2**qbits-1):
		complexArray.append(complex(0,0))

	sv = Statevector(complexArray)
	sv = sv.evolve(qc)

	outlistcomplex = sv.data.tolist()
	# print(outlistcomplex)
	# print (args)

	sigma = 0.0
	for i in range(len(outlistcomplex)):
		sigma  += abs( args[i].real - outlistcomplex[i].real)
		sigma  += abs( args[i].imag - outlistcomplex[i].imag)

	sigma = sigma/len(outlistcomplex)
	# print (sigma)
	return sigma

In [3]:
def guessrotations(target):
	# target is a statevector
	length = len(target)
	qbits = int(math.log2(length))
	# print (qbits)
	initial_guess = [1.0,] * 3*qbits
	# print(initial_guess)
	result = minimize(rotfitness, initial_guess, args=tuple(target))
	return result.x.tolist()

In [4]:
# List all the directories, only the directories
dirs = [d for d in os.listdir('.') if os.path.isdir(d)]
activeproj = {}
base = ""
for d in dirs:
	if os.path.exists(d+"/active"):
		with open(d+"/active","r") as f:
			ptype = f.read().strip()
			activeproj[d] = ptype
		if os.path.exists(d+"/base"):
			base = d
			if ptype != "qiskit":
				print("Warning: base project is not of type qiskit")
		
# print("Active projects: ",activeproj)
# print("Static data: ",staticdata)
# print("Base project: ",base)

In [5]:
def getRandomStateVectorWhole(qbits):
	sv=random_statevector(2**qbits)
	rot=guessrotations(sv)
	return sv,rot

In [6]:
def getRandomStateVectorRotations(qbits):
	states=[]
	rots=[]
	for i in range(qbits):
		# Random rotation 0 4pi
		theta = 2*math.pi*random.random()
		phi = 2 *math.pi*random.random()

		rot=[theta,phi,0]

		qc = QuantumCircuit(1)
		qc.rx(theta, 0)
		qc.ry(phi, 0)
	
		complexArray = []
		complexArray.append(complex(1,0))
		complexArray.append(complex(0,0))

		sv = Statevector(complexArray)
		sv = sv.evolve(qc)

		states.append(sv)
		rots.append(rot)

	rot=[]
	for i in range(qbits):
		rot+=rots[i]

	sv=states[0]
	for i in range(1,qbits):
		newsv=[]
		for j in range(len(sv.data)):
			for k in range(2):
				newsv.append(sv.data[j]*states[i].data[k])
		sv=Statevector(newsv)

	print(sv)
	return sv,rot

In [7]:
def getRandomStateVectorSingle(qbits):
	states=[]
	rots=[]
	for i in range(qbits):
		sv=random_statevector(2**qbits)
		# sv= Statevector([1/math.sqrt(2), 1j/math.sqrt(2)])
		# sv= Statevector([0, 1])
		states.append(sv)
		print ("State: ",sv)
		ai = sv.data[0].imag
		ar = sv.data[0].real
		bi = sv.data[1].imag
		br = sv.data[1].real
		d2 = br**2+ai**2
		print ("d2: ",d2)
		c2 = ai**2/(d2)
		print ("c2: ",c2)
		phi = 2.0*math.asin(math.sqrt(d2))
		print ("phi: ",phi)
		theta = 2.0* math.asin(math.sqrt(c2))
		print ("theta: ",theta)
		print ("rx:")
		print ( math.cos(theta/2), "-i",math.sin(theta/2))
		print ( "-i", math.sin(theta/2), math.cos(theta/2))

		print ("ry:")
		print ( math.cos(phi/2), "-",math.sin(phi/2))
		print ( math.sin(phi/2), math.cos(phi/2))

		g0ar = math.cos(theta/2)
		g0bi = math.sin(theta/2)
		print ("g0 = rx * 0")
		print (g0ar, "-i",g0bi)

		print ("g1 = ry * g0")
		

		rot = [theta,phi,0]
	# sv=random_statevector(2**qbits)
	# rot=guessrotations(sv)
	return sv,rot

# getRandomStateVector(2)

Create a set of inputs for the test cases. They should be normalized values, let's use the qiskit random_statevector function to generate them.

In [8]:
if fundamental:
	inputs = []
	rots = []
	outlist=[[1.0,0.0]]
	rotlist=[0.0,] * 3 * qbits
	for i in range(2**qbits-1):
		outlist.append([0.0,0.0])
	inputs.append(outlist)
	rots.append(rotlist)

	with open('inputs.json', 'w') as f:
		json.dump(inputs, f)

	with open('rots.json', 'w') as f:
		json.dump(rots, f)
else:
	if staticdata:
		with open('inputs.json', 'r') as f:
			inputs = json.load(f)
		with open('rots.json', 'r') as f:
			rots = json.load(f)
	else:
		inputs = []
		rots = []
		for i in range(numiter):
			sv,rot=getRandomStateVectorRotations(qbits)
			outlist=[]
			for i in range(len(sv)):
				outlist.append([sv[i].real,sv[i].imag])
			rots.append(rot)
			inputs.append(outlist)


		with open('inputs.json', 'w') as f:
			json.dump(inputs, f)

		with open('rots.json', 'w') as f:
			json.dump(rots, f)

Statevector([-0.05941508+0.52026434j, -0.26092027+0.36123422j,
             -0.55140171-0.04173932j, -0.39170284-0.26078378j],
            dims=(2, 2))
Statevector([-0.08276069-0.25415174j,  0.00117691+0.70522674j,
              0.05160807+0.22693332j,  0.05489897-0.61158434j],
            dims=(2, 2))
Statevector([-0.16016942+0.47435439j,  0.23075497+0.28220172j,
             -0.60495113-0.1921124j , -0.35239704+0.29898376j],
            dims=(2, 2))
Statevector([ 0.06086631+0.52562436j, -0.07318456+0.79899587j,
              0.07780628+0.13062117j,  0.07483529+0.21805484j],
            dims=(2, 2))
Statevector([-0.62115299+0.0122877j , -0.02666859-0.5602069j ,
              0.04898974-0.40324636j,  0.36510967+0.03409263j],
            dims=(2, 2))
Statevector([-0.18195335+0.12668302j,  0.42978382+0.37256724j,
              0.0053688 +0.28760143j,  0.71179771-0.19474219j],
            dims=(2, 2))
Statevector([-0.04082045-0.80476148j, -0.40508012-0.01327662j,
             -0.36255481+

Convert the inputs to the format expected by the single project.

In [9]:
for p in activeproj:
	if activeproj[p] == "qiskit" or activeproj[p] == "pennylane" or activeproj[p] == "spinqsim" or activeproj[p] == "spinq":
		inputsJson = []
		for i in inputs:
			inputJson=[]
			for j in i:
				val= [j[0], j[1]]
				inputJson.append(val)
			inputsJson.append( inputJson )

		with open(p+'/inputs.json', 'w') as f:
			json.dump(inputsJson, f)
		with open(p+'/rots.json', 'w') as f:
			json.dump(rots, f)

	if activeproj[p] == "bmqsim":
		inputsJson = []
		for i in inputs:
			inputJson=[]
			for j in i:
				val= {"Real": j[0], "Imag": j[1]}
				inputJson.append(val)
			inputsJson.append({ "Vector": inputJson })

		with open(p+'/inputs.json', 'w') as f:
			json.dump(inputsJson, f)
		with open(p+'/rots.json', 'w') as f:
			json.dump(rots, f)

	if activeproj[p] == "bmqsim_hw" or activeproj[p] == "quest":
		inputValues=[]
		for i in inputs:
			for j in i:
				inputValues.append(j[0])
				inputValues.append(j[1])

		with open(p+'/inputs.txt', 'w') as f:
			for item in inputValues:
				f.write("%s\n" % item)
		with open(p+'/rots.json', 'w') as f:
			json.dump(rots, f)
			

Run all the projects

In [10]:
for p in activeproj:	
	cwd = os.getcwd()
	os.chdir(p)
	if os.path.exists("circuit.py"):
		if activeproj[p] == "spinqsim" or activeproj[p] == "spinq":
			! source /tools/Conda/etc/profile.d/conda.sh && conda activate spinq && python circuit.py && conda deactivate
		elif activeproj[p] == "pennylane":
			! source /tools/Conda/etc/profile.d/conda.sh && conda activate pennylane && python circuit.py && conda deactivate
		else:
			%run circuit.py
	elif os.path.exists("circuit.ipynb"):
		%run circuit.ipynb
	os.chdir(cwd)

[{"Vector": [{"Real": -0.05941507664964196, "Imag": 0.5202643428423238}, {"Real": -0.2609202737044953, "Imag": 0.3612342158240593}, {"Real": -0.5514017142979879, "Imag": -0.04173931523276288}, {"Real": -0.391702839651354, "Imag": -0.26078377780014617}]}, {"Vector": [{"Real": -0.08276068604339133, "Imag": -0.25415174305624677}, {"Real": 0.0011769103366220252, "Imag": 0.705226735931841}, {"Real": 0.05160806915566047, "Imag": 0.22693331505739794}, {"Real": 0.05489897171689334, "Imag": -0.6115843415062158}]}, {"Vector": [{"Real": -0.16016942110820787, "Imag": 0.474354392055092}, {"Real": 0.23075497364845574, "Imag": 0.2822017176007854}, {"Real": -0.6049511261530719, "Imag": -0.19211239671018598}, {"Real": -0.35239703517137055, "Imag": 0.29898376477394006}]}, {"Vector": [{"Real": 0.06086631001970062, "Imag": 0.5256243633525698}, {"Real": -0.07318455884051453, "Imag": 0.7989958696833129}, {"Real": 0.07780628212896501, "Imag": 0.13062116518278932}, {"Real": 0.0748352907281873, "Imag": 0.21805

> Created connection with id 0
> On Fail Error Message:The opening handshake timed out
> Waiting for Reconnect...(1/3) Time out:  7   Login timeout!
> Waiting for Reconnect...(1/3) Time out:  4  > Created connection with id 1
> Waiting for Reconnect...(1/3) Time out:  0  
> Reconnected Create connection with id 2
> On Fail Error Message:The opening handshake timed out
> Waiting for Reconnect...(2/3) Time out:  17  > On Fail Error Message:The opening handshake timed out
> Waiting for Reconnect...(3/3) Time out:  11  Login timeout!
> Waiting for Reconnect...(3/3) Time out:  8   > Created connection with id 3
> Waiting for Reconnect...(3/3) Time out:  3  > On Fail Error Message:The opening handshake timed out
The maximum number of reconnections has been reached, and the program exits
Connect to server failed!
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
          ┌───┐
q_0: ─

OUTPUT checks the output of the projects and compares them with the pivot project.


In [11]:
passed = True
passwdProjects = {}
for p in activeproj:
	if (activeproj[p] == "qiskit" and p!=base) or activeproj[p] == "spinqsim" or activeproj[p] == "pennylane":
		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/outputs.json', 'r') as f:
			outputs = json.load(f)

		if len(qiskitoutputs) != len(outputs):
			print("Warning: Different number of outputs")

		qvars = []
		
		probs = []

		for i in range(len(qiskitoutputs[0])):
			qvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			spinqoutput = outputs[i]
			prob = []
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				qreal2 = float(spinqoutput[j][0])
				qimag2 = float(spinqoutput[j][1])
				if useprob:
					qvars[j][0] += pow(pow(qreal,2)+pow(qimag,2)-pow(qreal2,2)-pow(qimag2,2),2)
				else:
					qvars[j][0] += pow(qreal - qreal2, 2)
					qvars[j][1] += pow(qimag - qimag2, 2)

				prob.append(pow(qreal,2)+pow(qimag,2))
			probs.append(prob)

		with open(p+'/probs.json', 'w') as f:
			json.dump(probs, f)

		for i in range(len(qvars)):
			qvars[i][0] = pow(qvars[i][0]/len(qiskitoutputs), 0.5)
			qvars[i][1] = pow(qvars[i][1]/len(qiskitoutputs), 0.5)
			if qvars[i][0] > passthreshold or qvars[i][1] > passthreshold:
				passed = False
				passwdProjects[p] = False

		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(qvars, indent=4))
		
		if p not in passwdProjects:
			passwdProjects[p] = True

	if activeproj[p] == "spinq":
		if not useprob:
			panic("spinq only supports useprob mode")

		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/probs.json', 'r') as f:
			probs = json.load(f)

		if len(qiskitoutputs) != len(probs):
			print("Warning: Different number of outputs")

		qvars = []

		for i in range(len(qiskitoutputs[0])):
			qvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			spinqoutput = probs[i]
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				prob = float(spinqoutput[j])
				if useprob:
					qvars[j][0] += pow(pow(qreal,2)+pow(qimag,2)-prob,2)

		for i in range(len(qvars)):
			qvars[i][0] = pow(qvars[i][0]/len(qiskitoutputs), 0.5)
			qvars[i][1] = pow(qvars[i][1]/len(qiskitoutputs), 0.5)
			if qvars[i][0] > passthreshold or qvars[i][1] > passthreshold:
				passed = False
				passwdProjects[p] = False

		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(qvars, indent=4))

		if p not in passwdProjects:
			passwdProjects[p] = True
			
	if activeproj[p] == "bmqsim":
		
		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/outputs.json', 'r') as f:
			bmqsimoutputs = json.load(f)

		if len(qiskitoutputs) != len(bmqsimoutputs):
			print("Warning: Different number of outputs")

		bmvars = []
		probs = []
		for i in range(len(qiskitoutputs[0])):
			bmvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			bmqsimoutput = bmqsimoutputs[i]["Vector"]
			prob = []
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				breal = float(bmqsimoutput[j]["Real"])
				bimag = float(bmqsimoutput[j]["Imag"])
				if useprob:
					bmvars[j][0] += pow(pow(qreal,2)+pow(qimag,2)-pow(breal,2)-pow(bimag,2),2)
				else:
					bmvars[j][0] += pow(qreal - breal, 2)
					bmvars[j][1] += pow(qimag - bimag, 2)

				prob.append(pow(breal,2)+pow(bimag,2))
			probs.append(prob)

		with open(p+'/probs.json', 'w') as f:
			json.dump(probs, f)

		for i in range(len(bmvars)):
			bmvars[i][0] = pow(bmvars[i][0]/len(qiskitoutputs), 0.5)
			bmvars[i][1] = pow(bmvars[i][1]/len(qiskitoutputs), 0.5)
			if bmvars[i][0] > passthreshold or bmvars[i][1] > passthreshold:
				passed = False
				passwdProjects[p] = False

		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(bmvars, indent=4))

		if p not in passwdProjects:
			passwdProjects[p] = True

	if activeproj[p] == "bmqsim_hw" or activeproj[p] == "quest":
		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/outputs.txt', 'r') as f:
			bmqsimoutputs = []
			for k in range(len(qiskitoutputs)):
				bmqsimoutput=[]
				for i in range(2 ** qbits):
					line = f.readline()
					vreal=float(line)
					line = f.readline()
					vimm=float(line)
					bmqsimoutput.append([vreal,vimm])
				bmqsimoutputs.append(bmqsimoutput)

		if len(qiskitoutputs) != len(bmqsimoutputs):
			print("Warning: Different number of outputs")

		bmvars = []
		probs = []

		for i in range(len(qiskitoutputs[0])):
			bmvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			bmqsimoutput = bmqsimoutputs[i]
			prob = []
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				breal = float(bmqsimoutput[j][0])
				bimag = float(bmqsimoutput[j][1])
				if useprob:
					bmvars[j][0] += pow(pow(qreal,2)+pow(qimag,2)-pow(breal,2)-pow(bimag,2),2)
				else:
					bmvars[j][0] += pow(qreal - breal, 2)
					bmvars[j][1] += pow(qimag - bimag, 2)
				prob.append(pow(breal,2)+pow(bimag,2))
			probs.append(prob)

		for i in range(len(bmvars)):
			bmvars[i][0] = pow(bmvars[i][0]/len(qiskitoutputs), 0.5)
			bmvars[i][1] = pow(bmvars[i][1]/len(qiskitoutputs), 0.5)
			if bmvars[i][0] > passthreshold or bmvars[i][1] > passthreshold:
				passed = False
				passwdProjects[p] = False
		with open(p+'/probs.json', 'w') as f:
			json.dump(probs, f)
			
		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(bmvars, indent=4))

		if p not in passwdProjects:
			passwdProjects[p] = True

Average error on bmqsim:
[
    [
        2.549569407016236e-08,
        0.0
    ],
    [
        2.885251432307331e-08,
        0.0
    ],
    [
        3.692901511368905e-08,
        0.0
    ],
    [
        1.746272424388681e-08,
        0.0
    ]
]
Average error on spinqq:
[
    [
        0.3548942532525385,
        0.0
    ],
    [
        0.27781675777807957,
        0.0
    ],
    [
        0.3392762784300038,
        0.0
    ],
    [
        0.3514987275650321,
        0.0
    ]
]
Average error on bmqsim_hw:
[
    [
        2.632170705990516e-08,
        0.0
    ],
    [
        2.0258335698153443e-08,
        0.0
    ],
    [
        3.461590841562049e-08,
        0.0
    ],
    [
        1.5248231074959255e-08,
        0.0
    ]
]
Average error on pennylane:
[
    [
        2.6148317682333585e-17,
        0.0
    ],
    [
        1.2244862409847014e-17,
        0.0
    ],
    [
        7.290979246115659e-17,
        0.0
    ],
    [
        1.9687391937682635e-17,
        0.0


In [12]:
if not quiet:
	if passed:
		print(f"{CYAN}Overall{STANDARD}: {GREEN}Passed{STANDARD}")
	else:
		print(f"Overall: {RED}Failed{STANDARD}")
	print ("")
	print (f"{CYAN}Detailed results{STANDARD}:")
	for p in passwdProjects:
		if passwdProjects[p]:
			print(f"\t{MAGENTA}"+p+f"{STANDARD}"+f": {GREEN}Passed{STANDARD}")
		else:
			print(f"\t{MAGENTA}"+p+f"{STANDARD}"+f": {RED}Failed{STANDARD}")

Overall: Failed

Detailed results:
	bmqsim: Passed
	spinqq: Failed
	bmqsim_hw: Passed
	pennylane: Passed
	quest: Passed
	spinq: Passed
	bmqsim_alveo: Passed
	qiskitrot: Passed
